# Специализция вычислений функций Фибоначчи

## Пререквизиты

In [1]:
using Test
using MacroTools: prettify

## Числи Фибоначчи и числа "Гибоначчи"

In [2]:
fib(n) =
    if n == 0
        0
    elseif n == 1
        1
    else
        fib(n - 2) + fib(n - 1)
    end

fib (generic function with 1 method)

In [3]:
@test Tuple(fib(n) for n in 0:10) == (0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55)

Test Passed

Проблема в том, что у `fib` - только один аргумент. А если мы хотим специализировать функцию по значениям некоторых аргументов, то этих аргументов должно быть ≥ 2.

Посему, переходим к рассмотрению чисел "Гибоначчи".

In [4]:
gib(n, x, y) =
    if n == 0
        x
    elseif n == 1
        y
    else
        gib(n - 2, x, y) + gib(n - 1, x, y)
    end

gib (generic function with 1 method)

In [5]:
@test Tuple(gib(n, 0, 1) for n in 0:10) ==
      (0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55)
@test Tuple(gib(n, 2, 3) for n in 0:10) ==
      (2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233)

Test Passed

## Мемоизация

In [6]:
function gib_memo!(d, n, x, y)
    haskey(d, n) && return d[n]

    r =
        if n == 0
            x
        elseif n == 1
            y
        else
            gib_memo!(d, n - 2, x, y) + gib_memo!(d, n - 1, x, y)
        end

    @show n; d[n] = r
end

function gib_memo(n, x, y)
    d = Dict{Int64,Int64}()
    gib_memo!(d, n, x, y)
end

gib_memo (generic function with 1 method)

In [7]:
gib_memo(10, 2, 3)

n = 0
n = 1
n = 2
n = 3
n = 4
n = 5
n = 6
n = 7
n = 8
n = 9
n = 10


233

## Превращаем вычислитель в генератор

In [8]:
function gib_gen!(d, es, n)
    haskey(d, n) && return d[n]

    d[n] =
        if n == 0 (:x) elseif n == 1 (:y) else
            g = Symbol("g_", n)
            push!(es, :($g = $(gib_gen!(d, es, n - 2)) + $(gib_gen!(d, es, n - 1))))
            g
        end
end

function gib_gen(n)
    d = Dict{Int64,Any}()
    es = Expr[]
    r = gib_gen!(d, es, n)

    quote $(es...); $r end
end

gib_gen (generic function with 1 method)

In [9]:
gib_gen(0) |> prettify

:x

In [10]:
gib_gen(1) |> prettify

:y

In [11]:
gib_gen(3) |> prettify

quote
    g_2 = x + y
    g_3 = y + g_2
    g_3
end

In [12]:
gib_gen(10) |> prettify

quote
    g_2 = x + y
    g_3 = y + g_2
    g_4 = g_2 + g_3
    g_5 = g_3 + g_4
    g_6 = g_4 + g_5
    g_7 = g_5 + g_6
    g_8 = g_6 + g_7
    g_9 = g_7 + g_8
    g_10 = g_8 + g_9
    g_10
end

In [13]:
@generated function gib_gen(::Val{n}, x, y) where {n}
    gib_gen(n)
end

gib_gen (generic function with 2 methods)

In [14]:
@test Tuple(gib_gen(Val(n), 2, 3) for n in 0:10) ==
      (2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233)

Test Passed

## Использование комбинаторов

Потенциальная польза от этого может быть в том, что одно и то же определение функции может исполняться как с мемоизацией, так и без оной.

Или один и тот же механизм мемоизации может использоваться для разных функций.

In [15]:
fix(f, xs...) = f((ys...) -> fix(f, ys...), xs...)

fix (generic function with 1 method)

In [16]:
gib_step(self, n, x, y) =
    if n == 0
        x
    elseif n == 1
        y
    else
        self(n - 2, x, y) + self(n - 1, x, y)
    end

gib_step (generic function with 1 method)

In [17]:
@test Tuple(fix(gib_step, n, 2, 3) for n in 0:10) ==
      (2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233)

Test Passed

А зачем? А затем, что можно добавить а `fix` дополнительные действия, которые будут совершаться перед вызовом функции и после оного.

In [18]:
function fix_trace(f, xs...)
    println("Calling ", string(f), string(xs))
    r = f((ys...) -> fix_trace(f, ys...), xs...)
    println("Returning ", string(f), string(xs), " = ", r)
    return r
end

fix_trace (generic function with 1 method)

In [19]:
fix_trace(gib_step, 3, 2, 3)

Calling gib_step(3, 2, 3)
Calling gib_step(1, 2, 3)
Returning gib_step(1, 2, 3) = 3
Calling gib_step(2, 2, 3)
Calling gib_step(0, 2, 3)
Returning gib_step(0, 2, 3) = 2
Calling gib_step(1, 2, 3)
Returning gib_step(1, 2, 3) = 3
Returning gib_step(2, 2, 3) = 5
Returning gib_step(3, 2, 3) = 8


8

В частности, не меняя текст `fib_step`, можно подсунуть мемоизацию.

In [20]:
function fix_memo!(d, f, xs...)
    haskey(d, xs) && d[xs]
    d[xs] = f((ys...) -> fix_memo!(d, f, ys...), xs...)
end

fix_memo(f, xs...) = fix_memo!(Dict(), f, xs... )

fix_memo (generic function with 1 method)

In [21]:
@test Tuple(fix_memo(gib_step, n, 2, 3) for n in 0:10) ==
      (2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233)

Test Passed

## А можно ли определить `fix_trace` и `fix_memo` через стандартный `fix`?

(Непонятно, есть ли в этом практический смысл? Но, с точки зрения науки, - любопытно...)

In [22]:
function trace_step(self, f, xs...)
    println("Calling ", string(f), string(xs))
    r = f((ys...) -> self(f, ys...), xs...)
    println("Returning ", string(f), string(xs), " = ", r)
    return r
end

trace_step (generic function with 1 method)

In [23]:
fix(trace_step, gib_step, 3, 2, 3)

Calling gib_step(3, 2, 3)
Calling gib_step(1, 2, 3)
Returning gib_step(1, 2, 3) = 3
Calling gib_step(2, 2, 3)
Calling gib_step(0, 2, 3)
Returning gib_step(0, 2, 3) = 2
Calling gib_step(1, 2, 3)
Returning gib_step(1, 2, 3) = 3
Returning gib_step(2, 2, 3) = 5
Returning gib_step(3, 2, 3) = 8


8

И `fix_memo` можно переопределить через `fix`.

In [24]:
function mk_memo_step()
    d = Dict()

    (self, f, xs...) -> begin
        haskey(d, xs) && return d[xs]
        r = f((ys...) -> self(f, ys...), xs...)
        d[xs] = r
    end
end

mk_memo_step (generic function with 1 method)

In [25]:
@test Tuple(fix(mk_memo_step(), gib_step, n, 2, 3) for n in 0:10) ==
    (2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233)

Test Passed

## "Коронный номер": композиция двух "инструментовок"!

In [26]:
fix2(g, f) =
    (xs...) ->
        f((ys...) -> g((zs...) ->
                    fix2(g, f)(zs...), ys...), xs...)

fix2 (generic function with 1 method)

In [27]:
@test fix2(mk_memo_step(), trace_step)(gib_step, 3, 2, 3) == 8


Calling gib_step(3, 2, 3)
Calling gib_step(0, 2, 3)
Returning gib_step(0, 2, 3) = 2
Calling gib_step(1, 2, 3)
Returning gib_step(1, 2, 3) = 3
Returning gib_step(3, 2, 3) = 8


Test Passed